- 크롤링 2가지 
+ 여러페이지 크롤링
+ API 활용
  + JSON, XML

In [14]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd
from datetime import datetime 
import time

def get_code(company_code):
  url = 'https://finance.naver.com/item/main.naver?code=' + company_code
  response = requests.get(url)
  soup = BeautifulSoup(response.content, "html.parser")

  return soup

def get_price(company_code):
  soup = get_code(company_code)
  no_today = soup.select_one('p.no_today')
  # print(no_today)
  result = no_today.select_one('span.blind').text
  # 81,100 --> 숫자로 변경하는 코드 작성
  return result

def create_df(company_code, company_name, price_list):
  data = pd.DataFrame({
      '종목코드' : company_code, 
      '상장회사' : company_name, 
      '주가' : price_list
  })

  return data

if __name__ == '__main__':
  company_codes = ['035720', '005930', '030200']
  company_names = ['카카오', '삼성전자', 'KT']

  cur_time = datetime.now()
  print("-" * 60)
  print(cur_time)
  print("-" * 60)

  prices_list = []
  for code in company_codes:
    nowPrice = get_price(code)
    print("종목코드 :", code)
    print("현재주가 :", nowPrice) 
    prices_list.append(nowPrice)

  stock_df = create_df(company_codes, company_names, prices_list)
  print("-" * 60)
  print(stock_df)
  print("-" * 60)

------------------------------------------------------------
2022-08-08 00:50:45.566426
------------------------------------------------------------
종목코드 : 035720
현재주가 : 81,600
종목코드 : 005930
현재주가 : 60,900
종목코드 : 030200
현재주가 : 37,800
------------------------------------------------------------
     종목코드  상장회사      주가
0  035720   카카오  81,600
1  005930  삼성전자  60,900
2  030200    KT  37,800
------------------------------------------------------------


## 여러 페이지 가져오기

In [17]:
!pip install fake_useragent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=2f0a7d8e61abc4be88a048df72022a2a50a0b9c6ebd6beb9ea7a1f74b98aab38
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [20]:
from fake_useragent import UserAgent

company_code = '005930'
url = 'https://finance.naver.com/item/sise_day.nhn?code=' + company_code
ua = UserAgent()
headers = {
    'User-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
}
response= requests.get(url, headers = headers)
#print(response)

soup = BeautifulSoup(response.content, 'html.parser')
last_page = int(soup.select_one('td.pgRR').a['href'].split('=')[-1])
print(last_page)
print(type(last_page))

df = None
count = 0
for page in range(1, last_page +1):
  req = requests.get(f'{url}&page={page}', headers = headers)
  df = pd.concat([df, pd.read_html(req.text, encoding = "euc-kr")[0]], ignore_index=True)
  if count > 10:
    break
  count += 1

  time.sleep(5)
  
  df.dropna(inplace=True)
  df.reset_index(drop=True, inplace=True)


656
<class 'int'>


In [21]:
df

,날짜,종가,전일비,시가,고가,저가,거래량
0,2022.08.08,60900.0,600.0,61400.0,61400.0,60600.0,6134456.0
1,2022.08.05,61500.0,0.0,61700.0,61900.0,61200.0,9567620.0
2,2022.08.04,61500.0,200.0,61700.0,61800.0,61200.0,9125439.0
3,2022.08.03,61300.0,400.0,61600.0,61600.0,61000.0,10053861.0
4,2022.08.02,61700.0,400.0,61200.0,61900.0,61000.0,13614895.0
...,...,...,...,...,...,...,...
120,2022.02.18,74300.0,700.0,74600.0,74800.0,73700.0,10122226.0
121,2022.02.17,75000.0,200.0,75000.0,75600.0,74500.0,13605382.0
122,2022.02.16,74800.0,1100.0,74600.0,74900.0,74200.0,10767386.0
123,2022.02.15,73700.0,0.0,74100.0,74200.0,73100.0,9320365.0


In [22]:
import requests 

url = "http://data.ex.co.kr/openapi/trtm/realUnitTrtm?key=1308370912&type=json&iStartUnitCode=101&iEndUnitCode=103"
response = requests.get(url)
print(response)

<Response [200]>


In [23]:
json = response.json()
json

{'code': 'SUCCESS',
 'count': 544,
 'message': '인증키가 유효합니다.',
 'numOfRows': 10,
 'pageNo': 1,
 'pageSize': 55,
 'realUnitTrtmVO': [{'efcvTrfl': '229',
   'endUnitCode': '103 ',
   'endUnitNm': '수원신갈',
   'iEndUnitCode': None,
   'iStartEndStdTypeCode': None,
   'iStartUnitCode': None,
   'numOfRows': None,
   'pageNo': None,
   'startEndStdTypeCode': '2',
   'startEndStdTypeNm': '도착기준통행시간',
   'startUnitCode': '101 ',
   'startUnitNm': '서울',
   'stdDate': '20220808',
   'stdTime': '00  ',
   'sumTmUnitTypeCode': None,
   'tcsCarTypeCode': '1',
   'tcsCarTypeDivCode': '1',
   'tcsCarTypeDivName': '소형차',
   'tcsCarTypeName': '1종',
   'timeAvg': '7.883333333333334',
   'timeMax': '11.433',
   'timeMin': '5.750'},
  {'efcvTrfl': '94',
   'endUnitCode': '103 ',
   'endUnitNm': '수원신갈',
   'iEndUnitCode': None,
   'iStartEndStdTypeCode': None,
   'iStartUnitCode': None,
   'numOfRows': None,
   'pageNo': None,
   'startEndStdTypeCode': '2',
   'startEndStdTypeNm': '도착기준통행시간',
   'startUnitCod

In [24]:
cars = json ['realUnitTrtmVO']
records = []
for car in cars:
  dic = {}
  dic['date'] = car['stdDate']
  dic['time'] = car['stdTime']
  dic['timeAvg'] = car['timeAvg']
  records.append(dic)

  records

In [25]:
df = pd.DataFrame(records)
df

,date,time,timeAvg
0,20220808,00,7.883333333333334
1,20220808,00:00,8.016666666666667
2,20220808,00:05,8.0
3,20220808,00:10,8.05
4,20220808,00:15,7.975
5,20220808,00:20,7.983333333333333
6,20220808,00:25,7.8
7,20220808,00:30,8.025
8,20220808,00:35,8.1
9,20220808,00:40,7.95


In [26]:
import lxml
import requests
from bs4 import BeautifulSoup

url = "http://data.ex.co.kr/openapi/trtm/realUnitTrtm?key=1308370912&type=xml&iStartUnitCode=101&iEndUnitCode=103"
result = requests.get(url)
content = result.content
soup = BeautifulSoup(content, "lxml") # html.parser 
# print(soup)
print(soup.prettify())

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<html>
 <body>
  <data>
   <code>
    SUCCESS
   </code>
   <message>
    인증키가 유효합니다.
   </message>
   <count>
    610
   </count>
   <numofrows>
    10
   </numofrows>
   <pageno>
    1
   </pageno>
   <pagesize>
    61
   </pagesize>
   <realunittrtmvo>
    <efcvtrfl>
     19
    </efcvtrfl>
    <endunitcode>
     103
    </endunitcode>
    <endunitnm>
     수원신갈
    </endunitnm>
    <startendstdtypecode>
     2
    </startendstdtypecode>
    <startendstdtypenm>
     도착기준통행시간
    </startendstdtypenm>
    <startunitcode>
     101
    </startunitcode>
    <startunitnm>
     서울
    </startunitnm>
    <stddate>
     20220808
    </stddate>
    <stdtime>
     00:20
    </stdtime>
    <tcscartypecode>
     1
    </tcscartypecode>
    <tcscartypedivcode>
     1
    </tcscartypedivcode>
    <tcscartypedivname>
     소형차
    </tcscartypedivname>
    <tcscartypename>
     1종
    </tcscartypename>
    <timeavg>
     7.983333333333333
    </t

In [27]:
timeAvg_lists = soup.find_all("timeavg")
stddate_lists = soup.find_all("stddate")
stdtime_lists = soup.find_all("stdtime")

In [28]:
timeAvg_lists

[<timeavg>7.983333333333333</timeavg>,
 <timeavg>7.975</timeavg>,
 <timeavg>8.0</timeavg>,
 <timeavg>7.883333333333334</timeavg>,
 <timeavg>8.016666666666667</timeavg>,
 <timeavg>8.05</timeavg>,
 <timeavg>7.8</timeavg>,
 <timeavg>8.025</timeavg>,
 <timeavg>8.1</timeavg>,
 <timeavg>7.95</timeavg>]

In [31]:
avgTime = []
stddate = []
stdtime = []
for timeAvg, date, time in zip(timeAvg_lists, stddate_lists, stdtime_lists):
  # print(timeAvg, date, time)
  # print(time.get_text())
  avgTime.append(timeAvg.get_text())
  stddate.append(date.get_text())
  stdtime.append(time.get_text())

  df = pd.DataFrame({
      "date": stddate,
      "time": stdtime,
      "avgTime": avgTime
  })

  df